# Preprocesamiento de Neuroimágenes MRI y PET

### Proyecto: Monitorización Multimodal de Alzheimer  
#### Subfase 2: Preprocesamiento específico para neuroimágenes (MRI y PET) 
#### Pipeline de Preprocesamiento de Neuroimágenes  

| **Autor:** Abraham Tartalos  |   
**Versión del Notebook:** 1.0



## 📌 Descripción  
Pipeline automatizado para el preprocesamiento de imágenes estructurales (MRI) y funcionales (PET) en estudios de enfermedad de Alzheimer. Incluye:




## 🗂 Estructura de Directorios  
```bash
data/                           # Datos utilizados en el proyecto  
├── raw/                        # Datos sin procesar (originales)  
│   └── adni/                   # Dataset ADNI (o nombre de tu fuente)  
│       ├── images/             # Imágenes crudas  
│       │   ├── MRI/            # [DICOM/NIfTI] Imágenes crudas estructurales  
│       │   └── PET/            # [DICOM/NIfTI] Imágenes crudas funcionales  
│       └── metadata/           # (Opcional) Archivos clínicos/parámetros 
└── processed/
    ├── mri/          # Imágenes MRI preprocesadas
    ├── pet/          # Imágenes PET preprocesadas  
    ├── roi/          # Máscaras de regiones de interés
    └── reports/      # Reportes de calidad

***

## Importamos bibliotecas necesarias

In [2]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pydicom
import nibabel as nib
from datetime import datetime
from scipy import ndimage
from skimage import measure, filters, morphology, segmentation, exposure
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
from concurrent.futures import ProcessPoolExecutor, as_completed
from dask_image.ndfilters import gaussian_filter as dask_gaussian
import dask.array as da
import warnings


warnings.filterwarnings('ignore')

In [3]:
# Opcional: bibliotecas más específicas para neuroimágenes
try:
    import SimpleITK as sitk  # Para registro y filtrado
    import ants  # Para registro avanzado y segmentación
    HAS_ADVANCED_LIBS = True
except ImportError:
    print("Aviso: Algunas bibliotecas avanzadas no están disponibles. Se usarán alternativas básicas.")
    HAS_ADVANCED_LIBS = False


Aviso: Algunas bibliotecas avanzadas no están disponibles. Se usarán alternativas básicas.


## 1. Configuración de entorno

In [4]:
print("Configurando entorno de trabajo...")

# Definición de directorios
BASE_DIR = Path("../data")
RAW_DIR = BASE_DIR / "raw"
PROCESSED_DIR = BASE_DIR / "processed"
MRI_DIR = RAW_DIR / "adni/images/MRI"
PET_DIR = RAW_DIR / "adni/images/PET"
OUTPUT_DIR_MRI = PROCESSED_DIR / "mri"
OUTPUT_DIR_PET = PROCESSED_DIR / "pet"

REPORT_DIR = Path("../reports")
REPORT_PET = REPORT_DIR / "pet" 
REPORT_MRI = REPORT_DIR / "mri"

# Creación de directorios de salida si no existen
for dir_path in [OUTPUT_DIR_MRI, OUTPUT_DIR_PET]:
    os.makedirs(dir_path, exist_ok=True)

print(f"Directorios de entrada:")
print(f"- MRI: {MRI_DIR}")
print(f"- PET: {PET_DIR}")
print(f"Directorios de salida:")
print(f"- MRI procesados: {OUTPUT_DIR_MRI}")
print(f"- PET procesados: {OUTPUT_DIR_PET}")

Configurando entorno de trabajo...
Directorios de entrada:
- MRI: ..\data\raw\adni\images\MRI
- PET: ..\data\raw\adni\images\PET
Directorios de salida:
- MRI procesados: ..\data\processed\mri
- PET procesados: ..\data\processed\pet


## 2. Funciones auxiliares para procesamiento de DICOM

In [5]:
def list_dicom_files(directory, pattern='*.dcm'):
    """Lista todos los archivos DICOM en un directorio y sus subdirectorios."""
    directory = Path(directory)
    all_files = list(directory.glob(f"**/{pattern}"))
    return all_files

def read_dicom_series(dicom_files):
    """Lee una serie de archivos DICOM y los convierte a un array 3D."""
    # Leer todos los archivos DICOM
    dicoms = [pydicom.dcmread(str(f)) for f in dicom_files]
    
    # Ordenar por posición (SliceLocation o InstanceNumber)
    if hasattr(dicoms[0], 'SliceLocation'):
        dicoms.sort(key=lambda x: x.SliceLocation)
    else:
        dicoms.sort(key=lambda x: x.InstanceNumber)
    
    # Extraer pixeles y stack para formar un volumen 3D
    volume = np.stack([d.pixel_array for d in dicoms])
    
    # Normalizar a valores entre 0 y 1
    volume = volume.astype(np.float32)
    volume = (volume - volume.min()) / (volume.max() - volume.min())
    
    return volume, dicoms[0]

def convert_dicom_to_nifti(dicom_files, output_path):
    """Convierte archivos DICOM a formato NIfTI."""
    if HAS_ADVANCED_LIBS:
        # Usar SimpleITK para conversión más robusta
        reader = sitk.ImageSeriesReader()
        reader.SetFileNames([str(f) for f in dicom_files])
        image = reader.Execute()
        sitk.WriteImage(image, str(output_path))
    else:
        # Alternativa básica usando solo numpy y nibabel
        volume, first_dicom = read_dicom_series(dicom_files)
        
        # Crear affine transform básica (esto debería mejorarse en producción)
        pixel_spacing = first_dicom.PixelSpacing
        slice_thickness = first_dicom.SliceThickness if hasattr(first_dicom, 'SliceThickness') else 1.0
        
        affine = np.eye(4)
        affine[0, 0] = pixel_spacing[0]
        affine[1, 1] = pixel_spacing[1]
        affine[2, 2] = slice_thickness
        
        # Guardar como NIfTI
        nifti_img = nib.Nifti1Image(volume, affine)
        nib.save(nifti_img, output_path)
    
    return output_path

def preprocess_brain_volume(volume):
    """Preprocesamiento básico para volúmenes cerebrales."""
    # 1. Normalización de intensidad
    volume = exposure.rescale_intensity(volume)
    
    # 2. Eliminación de ruido con filtro gaussiano
    volume = ndimage.gaussian_filter(volume, sigma=1.0)
    
    # 3. Umbralización para separar cerebro del fondo (método simple)
    threshold = filters.threshold_otsu(volume)
    mask = volume > threshold
    
    # 4. Operaciones morfológicas para limpiar la máscara
    mask = morphology.binary_opening(mask, morphology.ball(2))
    mask = morphology.binary_closing(mask, morphology.ball(2))
    
    # 5. Etiquetado de componentes y selección del componente más grande (el cerebro)
    labels = measure.label(mask)
    regions = measure.regionprops(labels)
    if regions:
        largest_region = max(regions, key=lambda r: r.area)
        mask = labels == largest_region.label
    
    # 6. Aplicar máscara al volumen original
    volume_masked = volume.copy()
    volume_masked[~mask] = 0
    
    return volume_masked

def register_to_template(image_path, template_path, output_path):
    """Registra una imagen a una plantilla estándar."""
    if not HAS_ADVANCED_LIBS:
        print("Registro espacial requiere ANTs o SimpleITK. Saltando este paso.")
        return image_path
    
    try:
        # Usando ANTs para registro de alta calidad
        moving_image = ants.image_read(str(image_path))
        fixed_image = ants.image_read(str(template_path))
        
        # Registro
        registration = ants.registration(
            fixed=fixed_image,
            moving=moving_image,
            type_of_transform='SyN'  # SyN para registro no lineal
        )
        
        # Guardar resultado
        registered_image = registration['warpedmovout']
        ants.image_write(registered_image, str(output_path))
        
        return output_path
    except Exception as e:
        print(f"Error en registro: {e}")
        return image_path

def extract_brain_features(image_path, mask_path=None):
    """Extrae características relevantes de neuroimágenes para Alzheimer."""
    # Cargar imagen
    if str(image_path).endswith('.nii') or str(image_path).endswith('.nii.gz'):
        img = nib.load(str(image_path))
        data = img.get_fdata()
    else:
        # Asumimos que es una serie DICOM
        dicom_files = list_dicom_files(image_path) if os.path.isdir(image_path) else [image_path]
        data, _ = read_dicom_series(dicom_files)
    
    # Cargar máscara si existe
    if mask_path:
        mask_img = nib.load(str(mask_path))
        mask = mask_img.get_fdata() > 0
    else:
        # Crear máscara simple si no se proporciona
        threshold = filters.threshold_otsu(data)
        mask = data > threshold
    
    # Características básicas de la imagen
    features = {}
    
    # 1. Estadísticas básicas por región
    regions = {
        'whole_brain': mask,
        # Aquí se añadirían máscaras para regiones específicas como hipocampo, etc.
    }
    
    for region_name, region_mask in regions.items():
        region_data = data[region_mask]
        if len(region_data) > 0:
            features[f"{region_name}_mean"] = np.mean(region_data)
            features[f"{region_name}_std"] = np.std(region_data)
            features[f"{region_name}_median"] = np.median(region_data)
            features[f"{region_name}_min"] = np.min(region_data)
            features[f"{region_name}_max"] = np.max(region_data)
            features[f"{region_name}_volume"] = np.sum(region_mask)
    
    # 2. Características de textura (GLCM simplificado)
    # En producción usar librería como radiomics para características más avanzadas
    from skimage.feature import graycomatrix, graycoprops
    
    # Tomar una slice central representativa
    central_slice = data[:, :, data.shape[2]//2]
    central_slice = exposure.rescale_intensity(central_slice, out_range=(0, 255)).astype(np.uint8)
    
    # Calcular GLCM
    try:
        glcm = graycomatrix(central_slice, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
        features['texture_contrast'] = graycoprops(glcm, 'contrast')[0, 0]
        features['texture_dissimilarity'] = graycoprops(glcm, 'dissimilarity')[0, 0]
        features['texture_homogeneity'] = graycoprops(glcm, 'homogeneity')[0, 0]
        features['texture_energy'] = graycoprops(glcm, 'energy')[0, 0]
        features['texture_correlation'] = graycoprops(glcm, 'correlation')[0, 0]
    except Exception as e:
        print(f"Error al calcular características de textura: {e}")
    
    return features

## 3. Preprocesamiento de imágenes MRI

In [6]:
print("\nIniciando preprocesamiento de imágenes MRI...")

# Listar todos los archivos MRI DICOM
mri_files = list_dicom_files(MRI_DIR)
print(f"Se encontraron {len(mri_files)} archivos DICOM de MRI.")


# Listar todos los archivos MRI DICOM verdaderos
all_candidates = list_dicom_files(MRI_DIR)
mri_files = [f for f in all_candidates if f.suffix.lower()=='.dcm' and f.is_file()]
print(f"Se encontraron {len(mri_files)} archivos DICOM de MRI válidos.")


# Agrupar por serie/paciente (simplificado - en producción hacer más robusto)
mri_groups = {}
for file_path in mri_files:
    try:
        dicom = pydicom.dcmread(str(file_path))
        # Usar combinación de ID de paciente y ID de serie como clave
        if hasattr(dicom, 'PatientID') and hasattr(dicom, 'SeriesInstanceUID'):
            key = f"{dicom.PatientID}_{dicom.SeriesInstanceUID}"
            if key not in mri_groups:
                mri_groups[key] = []
            mri_groups[key].append(file_path)
    except Exception as e:
        print(f"Error al leer {file_path}: {e}")

print(f"Se identificaron {len(mri_groups)} series MRI distintas.")

print("=="*30)
# Mostrar el total de grupos disponibles
total_grupos = len(mri_groups)
print(f"Total de grupos disponibles: {total_grupos}")
print("=="*30)

# Procesar cada serie MRI
mri_features = []
for i, (group_key, group_files) in enumerate(tqdm(list(mri_groups.items())[:-1])):  # Procesar solo 5 para ejemplo
    try:
        # 1. Extraer metadatos del primer archivo
        dicom = pydicom.dcmread(str(group_files[0]))
        patient_id = dicom.PatientID if hasattr(dicom, 'PatientID') else f"unknown_patient_{i}"
        
        # 2. Convertir a NIfTI para procesamiento más fácil
        nifti_path = OUTPUT_DIR_MRI / f"{patient_id}_raw.nii.gz"
        convert_dicom_to_nifti(group_files, nifti_path)
        
        # 3. Preprocesamiento básico
        # Cargar volumen
        img = nib.load(nifti_path)
        data = img.get_fdata()
        
        # Aplicar preprocesamiento
        processed_data = preprocess_brain_volume(data)
        
        # Guardar resultado
        processed_path = OUTPUT_DIR_MRI / f"{patient_id}_processed.nii.gz"
        processed_img = nib.Nifti1Image(processed_data, img.affine)
        nib.save(processed_img, processed_path)
        
        # 4. Registro a plantilla estándar (opcional, descomentar si necesario)
        # template_path = Path("./templates/mni152_t1_1mm.nii.gz")  # Ajustar ruta a tu plantilla
        # if template_path.exists():
        #     registered_path = OUTPUT_DIR_MRI / f"{patient_id}_registered.nii.gz"
        #     register_to_template(processed_path, template_path, registered_path)
        
        # 5. Extracción de características
        features = extract_brain_features(processed_path)
        features['patient_id'] = patient_id
        features['modality'] = 'MRI'
        features['file_path'] = str(processed_path)
        
        mri_features.append(features)
        
    except Exception as e:
        print(f"Error procesando grupo {group_key}: {e}")

# Convertir a DataFrame
if mri_features:
    mri_features_df = pd.DataFrame(mri_features)
    mri_features_df.to_csv(OUTPUT_DIR_MRI / "mri_features.csv", index=False)
    print(f"Se extrajeron características de {len(mri_features)} imágenes MRI.")
    display(mri_features_df.head())
else:
    print("No se pudieron extraer características de las imágenes MRI.")



Iniciando preprocesamiento de imágenes MRI...
Se encontraron 1672 archivos DICOM de MRI.
Se encontraron 1195 archivos DICOM de MRI válidos.
Se identificaron 4 series MRI distintas.
Total de grupos disponibles: 4


  0%|          | 0/3 [00:00<?, ?it/s]

Se extrajeron características de 3 imágenes MRI.


,whole_brain_mean,whole_brain_std,whole_brain_median,whole_brain_min,whole_brain_max,whole_brain_volume,texture_contrast,texture_dissimilarity,texture_homogeneity,texture_energy,texture_correlation,patient_id,modality,file_path
0,0.020042,0.008530,0.017690,0.009782,0.200336,3863045,61.098023,2.853322,0.747431,0.712147,0.974330,082_S_4224,MRI,..\data\processed\mri\082_S_4224_processed.nii.gz
1,0.049783,0.020518,0.045161,0.025124,0.313711,40304,129.713729,3.718741,0.800853,0.785883,0.931925,082_S_7117,MRI,..\data\processed\mri\082_S_7117_processed.nii.gz
2,0.024835,0.010895,0.021900,0.011967,0.291776,4283401,16.729921,1.198351,0.843343,0.800366,0.972220,130_S_5175,MRI,..\data\processed\mri\130_S_5175_processed.nii.gz


## 4. Preprocesamiento de imágenes PET

In [ ]:
print("\nIniciando preprocesamiento de imágenes PET (subconjunto reducido)…")

# 1) Identificar carpetas de serie que contengan .dcm
series_dirs = []
for root, dirs, files in os.walk(PET_DIR):
    if any(f.lower().endswith('.dcm') for f in files):
        series_dirs.append(Path(root))
print(f"Encontradas {len(series_dirs)} carpetas de serie PET.")

# 2) Agrupar sólo series completas
pet_groups = {}
for sd in series_dirs:
    dcm_files = [f for f in sd.glob("*.dcm") if f.is_file()]
    if len(dcm_files) < 10:
        continue
    try:
        ds = pydicom.dcmread(str(dcm_files[0]), stop_before_pixels=True)
        if getattr(ds, 'Modality','').upper() == 'PT':
            key = f"{ds.PatientID}_{ds.SeriesInstanceUID}"
            pet_groups[key] = dcm_files
    except Exception:
        continue
print(f"Preparadas {len(pet_groups)} series PET en total.")

# 3) Seleccionar un subconjunto de entre 5 y 10 series
max_subset = 5  # número máximo de series a procesar
selected_keys = list(pet_groups.keys())[:max_subset]
pet_groups = {k: pet_groups[k] for k in selected_keys}
print(f"Procesando subconjunto de {len(pet_groups)} series PET para prueba.")

# Parámetros de procesamiento paralelo\
max_workers = 1  # ajusta según tu CPU

# 4) Función de procesamiento de una sola serie (similar al pipeline original)
def process_pet_series(args):
    key, files = args
    start = datetime.now()
    try:
        ds = pydicom.dcmread(str(files[0]), stop_before_pixels=True)
        patient_id = ds.PatientID
        # Conversión DICOM -> NIfTI
        raw_nifti = OUTPUT_DIR_PET / f"{patient_id}_raw.nii.gz"
        if not raw_nifti.exists():
            convert_dicom_to_nifti(files, raw_nifti)
        # Carga y preprocesamiento
        img = nib.load(raw_nifti)
        data = img.get_fdata()
        # Normalización simplificada y suavizado gaussiano
        proc_data = data / np.mean(data[data > 0])
        proc_data = ndimage.gaussian_filter(proc_data, sigma=1.0)
        processed_nifti = OUTPUT_DIR_PET / f"{patient_id}_processed.nii.gz"
        nib.save(nib.Nifti1Image(proc_data, img.affine), processed_nifti)
        # Extracción de features
        feats = extract_brain_features(processed_nifti)
        feats.update({'patient_id': patient_id,
                      'modality': 'PET',
                      'file_path': str(processed_nifti)})
        elapsed = (datetime.now() - start).total_seconds()
        print(f"Serie {key} procesada en {elapsed:.1f} seg.")
        return feats
    except Exception as e:
        elapsed = (datetime.now() - start).total_seconds()
        print(f"Error en serie {key} tras {elapsed:.1f} seg: {e}")
        return None

# 5) Procesamiento paralelo del subconjunto
pet_features = []
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(process_pet_series, item): item for item in pet_groups.items()}
    for fut in tqdm(as_completed(futures), total=len(futures), desc="Series PET procesadas"):
        res = fut.result()
        if res:
            pet_features.append(res)

# 6) Guardar resultados
if pet_features:
    pet_df = pd.DataFrame(pet_features)
    pet_df.to_csv(OUTPUT_DIR_PET / "pet_features_subset.csv", index=False)
    print(f"Se extrajeron características de {len(pet_features)} series PET.")
    display(pet_df.head())
else:
    print("No se pudieron extraer características del subconjunto PET.")


Iniciando preprocesamiento de imágenes PET (subconjunto reducido)…
Encontradas 360 carpetas de serie PET.
Preparadas 293 series PET en total.
Procesando subconjunto de 5 series PET para prueba.


Series PET procesadas:   0%|          | 0/5 [00:00<?, ?it/s]

## 5. Visualización de resultados del preprocesamiento

In [11]:
print("\nGenerando visualizaciones de control de calidad...")

def plot_preprocessing_results(original_path, processed_path, title):
    """Muestra comparación de imagen original y procesada."""
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Cargar imágenes
    original_img = nib.load(original_path)
    original_data = original_img.get_fdata()
    
    processed_img = nib.load(processed_path)
    processed_data = processed_img.get_fdata()
    
    # Mostrar slices centrales
    slice_idx = original_data.shape[2] // 2
    
    axes[0].imshow(original_data[:, :, slice_idx], cmap='gray')
    axes[0].set_title('Original')
    axes[0].axis('off')
    
    axes[1].imshow(processed_data[:, :, slice_idx], cmap='gray')
    axes[1].set_title('Procesada')
    axes[1].axis('off')
    
    plt.suptitle(title)
    plt.tight_layout()
    return fig

# Visualizar un ejemplo de MRI procesado (si existe)
mri_examples = list(OUTPUT_DIR_MRI.glob('*_raw.nii.gz'))
if mri_examples:
    original_path = mri_examples[0]
    processed_path = original_path.parent / original_path.name.replace('_raw', '_processed')
    if processed_path.exists():
        fig = plot_preprocessing_results(original_path, processed_path, 'Ejemplo de Preprocesamiento MRI')
        plt.savefig(REPORT_MRI / 'mri_preprocessing_example.png')
        plt.close(fig)

# Visualizar un ejemplo de PET procesado (si existe)
pet_examples = list(OUTPUT_DIR_PET.glob('*_raw.nii.gz'))
if pet_examples:
    original_path = pet_examples[0]
    processed_path = original_path.parent / original_path.name.replace('_raw', '_processed')
    if processed_path.exists():
        fig = plot_preprocessing_results(original_path, processed_path, 'Ejemplo de Preprocesamiento PET')
        plt.savefig(REPORT_PET / 'pet_preprocessing_example.png')
        plt.close(fig)



Generando visualizaciones de control de calidad...


## 6. Integración de características MRI y PET

In [12]:
print("\nIntegrando características de MRI y PET...")

# Combinar características si existen ambos tipos
if 'mri_features_df' in locals() and 'pet_features_df' in locals():
    print("1")
    # Concatenar
    all_features_df = pd.concat([mri_features_df, pet_features_df], ignore_index=True)
    
    # Guardar
    all_features_df.to_csv(PROCESSED_DIR / "neuroimaging_features.csv", index=False)
    print(f"Se guardaron {len(all_features_df)} registros de características de neuroimágenes.")
    
    # Pivot table para ver pacientes con ambas modalidades
    modality_pivot = all_features_df.pivot_table(
        index='patient_id', 
        columns='modality', 
        values='file_path', 
        aggfunc='count', 
        fill_value=0
    )
    
    print("\nDistribución de modalidades por paciente:")
    display(modality_pivot.head())
    
    # Estadísticas de características por modalidad
    print("\nEstadísticas de características por modalidad:")
    numeric_features = all_features_df.select_dtypes(include=[np.number])
    stats_by_modality = all_features_df.groupby('modality')[numeric_features.columns].mean()
    display(stats_by_modality)
else:
    print("2")


Integrando características de MRI y PET...
2


## 7. Resumen y recomendaciones para siguiente fase

In [13]:
print("\n" + "="*80)
print("RESUMEN DE PREPROCESAMIENTO DE NEUROIMÁGENES")
print("="*80)

print("""
Se ha completado el preprocesamiento básico de neuroimágenes MRI y PET:

1. Conversión de formato DICOM a NIfTI para facilitar análisis
2. Preprocesamiento específico por modalidad:
   - MRI: Normalización, reducción de ruido, segmentación cerebral
   - PET: Normalización específica (pseudo-SUV), suavizado
3. Extracción de características básicas:
   - Estadísticas por región cerebral
   - Características de textura
4. Integración de características de ambas modalidades

Limitaciones actuales:
- No se ha implementado registro completo a atlas estándar MNI152
- Segmentación básica de regiones cerebrales (sin atlas específico de regiones)
- Características de textura simplificadas

Recomendaciones para siguiente fase (Feature Engineering):
1. Implementar segmentación avanzada de regiones específicas para Alzheimer:
   - Hipocampo
   - Corteza entorrinal
   - Volumen ventricular
   - Grosor cortical
2. Calcular biomarcadores específicos:
   - MRI: Volumetría de regiones, índices de atrofia
   - PET: SUVr en regiones específicas comparadas con cerebelo
3. Incorporar atlas cerebrales para normalización espacial
4. Desarrollar características específicas para detección temprana de Alzheimer
""")

print("\nPreprocesamiento de neuroimágenes completado!")


RESUMEN DE PREPROCESAMIENTO DE NEUROIMÁGENES

Se ha completado el preprocesamiento básico de neuroimágenes MRI y PET:

1. Conversión de formato DICOM a NIfTI para facilitar análisis
2. Preprocesamiento específico por modalidad:
   - MRI: Normalización, reducción de ruido, segmentación cerebral
   - PET: Normalización específica (pseudo-SUV), suavizado
3. Extracción de características básicas:
   - Estadísticas por región cerebral
   - Características de textura
4. Integración de características de ambas modalidades

Limitaciones actuales:
- No se ha implementado registro completo a atlas estándar MNI152
- Segmentación básica de regiones cerebrales (sin atlas específico de regiones)
- Características de textura simplificadas

Recomendaciones para siguiente fase (Feature Engineering):
1. Implementar segmentación avanzada de regiones específicas para Alzheimer:
   - Hipocampo
   - Corteza entorrinal
   - Volumen ventricular
   - Grosor cortical
2. Calcular biomarcadores específicos:
   -

***

__Abraham Tartalos__